In [ ]:
%reload_ext autoreload
%autoreload 2

import os
import sys
import ast
import json
from collections import defaultdict
from matplotlib import pyplot as plt

import cv2
import numpy as np
import pandas as pd
import neuroglancer
from tqdm import tqdm
from skimage import io
import imagesize
from collections import OrderedDict

sys.path.append(os.path.join(os.getcwd(), '../'))
from utilities.FileLocationManager import FileLocationManager
from utilities.SqlController import SqlController
from utils import get_structure_number

In [ ]:
animal = 'DK52'
downsample_factor = 32
IMGDOWN32 = f'/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/{animal}/preps/CH1/thumbnail_aligned'
sqlController = SqlController(animal)
resolution = sqlController.scan_run.resolution
aligned_shape = np.array((sqlController.scan_run.width, sqlController.scan_run.height))
num_section = len(os.listdir(IMAGE_DIR_PATH))
downsampled_aligned_shape = np.round(aligned_shape / downsample_factor).astype(int)
scales = np.array([resolution, resolution, 20]) * 1000
downsampled_scales = np.array([resolution * downsample_factor, resolution * downsample_factor, 20]) * 1000
print('Full res scales', scales)
print('Downsampled scales', downsampled_scales)
print('aligned shape', aligned_shape)
print('downsampled aligned shape', downsampled_aligned_shape)

In [ ]:
num_section

## Get the COM of DK52 from CVAT

In [ ]:
DK52_centers = {'12N': [46488, 18778, 242],
                '5N_L': [38990, 20019, 172],
                '5N_R': [39184, 19027, 315],
                '7N_L': [42425, 23190, 166],
                '7N_R': [42286, 22901, 291]}
centers = OrderedDict(DK52_centers)
centers_list = []
for value in centers.values():
    centers_list.append((value[1], value[0], value[2]))
COM = np.array(centers_list)
print(COM)

In [ ]:
DIR='/net/birdstore/Active_Atlas_Data/data_root/atlas_data/atlasV7/origin'
for s in DK52_centers.keys():
    filepath = os.path.join(DIR, f'{s}.txt')
    arr = np.loadtxt(filepath)
    print(s,arr)


In [ ]:
font = cv2.FONT_HERSHEY_SIMPLEX
fontScale = 2
thickness = 5

In [ ]:
for structure,v in centers.items():
    color = sqlController.get_structure_color_rgb(structure)
    filename = str(v[2]) + '.tif'
    filepath = os.path.join(IMGDOWN32, filename)
    img = cv2.imread(filepath)
    cx = int(round(v[0]))
    cy = int(round(v[1]))
    cxdisplay = cx // downsample_factor
    cydisplay = cy // downsample_factor
    section = v[2]
    print(structure,section,'with offset',cx,cy)
    cv2.circle(img, (cxdisplay,cydisplay), 20, (0,200,0), -1)

    fig=plt.figure(figsize=(26,18), dpi= 100, facecolor='w', edgecolor='k')
    plt.imshow(img, cmap="gray")
    title = f'Center of mass {cx}x{cy} for structure {structure} at section {section}\n'
    subtitle = f'Downsampled by 1/{downsample_factor}'
    plt.title(title + subtitle, fontsize=30)
    plt.tick_params(axis='x', labelsize=30)
    plt.tick_params(axis='y', labelsize=30)
    plt.show()
    outpath = f'/net/birdstore/Active_Atlas_Data/data_root/atlas_data/DK52.{structure}.section.{section}.jpg'
    fig.savefig(outpath, bbox_inches='tight')

In [ ]:
def reverse_transform_create_alignment(points, transform):
    c = np.hstack((points, np.ones((points.shape[0], 1))))
    b = transform.copy()[:, 0:2] # Reverse rotation matrix by doing R^-1 = R^T
    b[2:, 0:2] = -transform[0:2, 2] # Reverse translation matrix by doing -T
    a = np.matmul(c, b)
    return a

In [ ]:
#162, 185, 210

file_name = '162.tif'
filepath = os.path.join(DOWN32, file_name)
img = cv2.imread(filepath)
section = int(file_name.split('.')[0])

for structure in section_structure_vertices[section]:
    pts = section_structure_vertices[section][structure]
    points = np.array(pts, dtype=np.int32)
    offset = section_offset[section]
    transform = section_transform[section]
    
    points = reverse_transform_create_alignment(points, section_transform[section]) # reverse create_alignment transform
    points = (points - section_offset[section]).astype(np.int32) # reverse create_clean offset
    
    
    cx, cy = np.mean(points, axis=0)
    #print(structure,section,'with centers',cx,cy, 'offset', offset)
    cv2.polylines(img, [points], isClosed=True, color=colors[structure], thickness=2)
    cv2.putText(img, structure, (int(cx),int(cy)), font,
                1, colors[structure], 1, cv2.LINE_AA)

fig=plt.figure(figsize=(26,18), dpi= 100, facecolor='w', edgecolor='k')
plt.imshow(img, cmap="gray")
plt.title('Pre alignment section:{}'.format(section), fontsize=30)
plt.tick_params(axis='x', labelsize=30)
plt.tick_params(axis='y', labelsize=30)
plt.show()

# Fill up a pandas dataframe with the corrected vertices and save it.

In [ ]:
data = []

files = sorted(os.listdir(DOWN32))
for file_name in files:
    section = int(file_name.split('.')[0])
    
    if section in section_structure_vertices:
        for structure in section_structure_vertices[section]:
            pts = section_structure_vertices[section][structure]
            points = np.array(pts, dtype=np.int32)
            points = reverse_transform_create_alignment(points, section_transform[section]) # reverse create_alignment transform
            points = points - section_offset[section] # reverse create_clean offset
            data.append([structure, section, points])
            
df = pd.DataFrame(data=data, columns=['structure', 'section', 'vertices'])
outpath = os.path.join(CSV_DIR_PATH, f'{animal}_sections.162.185.210.csv')
df.to_csv(outpath, index=False)
redone_vertices = defaultdict(dict)
for index,row in df.iterrows():
    section = row['section']
    structure = row['structure']
    points = row['vertices']
    redone_vertices[section][structure] = points

In [ ]:
sections = [162, 185, 210]

font = cv2.FONT_HERSHEY_SIMPLEX
fontScale = 1
file_name = '210.tif'
filepath = os.path.join(DOWN32, file_name)
img = cv2.imread(filepath)
section = int(file_name.split('.')[0])

sl = []
for structure in section_structure_vertices[section]:
    pts = redone_vertices[section][structure]
    points = np.array(pts, dtype=np.int32)
    cx, cy = np.mean(points, axis=0)
    sl.append(structure)
    #print(structure,section,'with centers',cx,cy, 'offset', offset)
    cv2.polylines(img, [points], isClosed=True, color=colors[structure], thickness=2)
    cv2.putText(img, structure, (int(cx-5),int(cy-5)), font,
                1, colors[structure], 1, cv2.LINE_AA)

fig=plt.figure(figsize=(26,18), dpi= 100, facecolor='w', edgecolor='k')
plt.imshow(img, cmap="gray")
plt.title('MD589 at 1/32 size section:{}, structures {}'.format(section, sl), fontsize=20)
plt.tick_params(axis='x', labelsize=30)
plt.tick_params(axis='y', labelsize=30)
plt.show()
fig.savefig(f'/home/eddyod/Desktop/MD589.section{section}.jpg', bbox_inches='tight')